# Working with Pandas

![](https://github.com/spatialthoughts/python-foundation-web/blob/master/images/python_foundation/pandas-logo.png?raw=1)

Pandas is a powerful library for working with data. Pandas provides fast and easy functions for reading data from files, and analyzing it.

Pandas is based on another library called `numpy` - which is widely used in scientific computing. Pandas extends `numpy` and provides new data types such as **Index**, **Series** and **DataFrames**.

Pandas implementation is very fast and efficient - so compared to other methods of data processing - using `pandas` results is simpler code and quick processing. We will now re-implement our code for reading a file and computing distance using Pandas.

## Downloading Data
To begin, we need to download worldcities.csv and place it in a folder called 'data' in our working directory. Our working directory is whereever this notebook is stored. We'll use a OneDrive share link, so it will be best to use a module called onedrivedownloader. We'll first check to see if it is already installed, and if not, install it. Then we'll use it to download the file and put it in the 'data' folder. If the 'data' folder does not exist, it will be created.

In [ ]:
# 1. Check/Install onedrivedownloader
try:
    import onedrivedownloader
except ImportError:
    print("Installing onedrivedownloader...")
    %pip install onedrivedownloader
    import onedrivedownloader
print("✓ Module onedrivedownloader ready.")

# 2. Define the OneDrive share link to worldcities.csv
link = "https://etsu365-my.sharepoint.com/:x:/g/personal/ernenwei_etsu_edu/IQCRz_-B4mxsTphsD4yAULWfARzVrJqhbX2DEodjbBTv4Wg?e=7Xd2gT"
data_dir = "data"

# 3. Download the data
# This creates the 'data' folder automatically and respects existing files
print(f"Checking for data in '{data_dir}'...")
onedrivedownloader.download(link, filename=f"{data_dir}/", force_download=False)

# 4. Verify the result
import os
files = os.listdir(data_dir)
if files:
    print(f"✓ Success! Folder '{data_dir}' contains: {files}")
else:
    print(f"⚠ Warning: '{data_dir}' is empty. Check your OneDrive link permissions.")


## Reading Files

By convention, `pandas` is commonly imported as `pd`

In [ ]:
import pandas as pd

Here we set up the path to worldcities.csv

In [ ]:
data_folder = 'data'
filename = 'worldcities.csv'
path = os.path.join(data_folder, filename)
print(path)

A **DataFrame** is the most used Pandas object. You can think of a DataFrame being equivalent to a Spreadsheet or an Attribute Table of a GIS layer.

Pandas provide easy methods to directly read files into a DataFrame. You can use methods such as `read_csv()`, `read_excel()`, `read_hdf()` and so forth to read a variety of formats. Here we will read the `worldcitites.csv` file using `read_csv()` method.

In [ ]:
df = pd.read_csv(path)

In [ ]:
df

Once the file is read and a DataFrame object is created, we can inspect it using the `head()` method.

The `head()` method is a Pandas DataFrame method that displays the first 5 rows of the DataFrame. This is very useful for quickly inspecting the structure and content of your data after it has been loaded.

In [ ]:
df.head()

There is also a `info()` method that shows basic information about the dataframe, such as number of rows/columns and data types of each column.

In [ ]:
df.info()

## Filtering Data

Pandas have many ways of selecting and filtered data from a dataframe. We will now see how to use [Boolean Filtering](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) to filter the dataframe to rows that match a condition.

In [ ]:
# Select all the rows where 'country' column is 'India'
# Output results as 'filtered'
home_country = 'India'
filtered = df[df['country'] == home_country]
filtered

In [ ]:
# Select all rows where 'population' column is > 1 mil
# Output results as 'filtered'
# note - this overwrites the 'filtered' variable created in previous cell
filtered = df[df['population'] > 1000000]
filtered

Filtered dataframe is a just view of the original data and we cannot make changes to it. We can save the filtered view to a new dataframe using the `copy()` method.

In [ ]:
country_df = df[df['country'] == home_country].copy()

In [ ]:
# Filters df to where 'city' column is 'Mumbai'
# Use iloc[0] to select the first matching row using an index of 0,
# then extract the value from the 'lng' (longitude) column
# Basically, this code retrieves the longitude of Mumbai
home_city = 'Mumbai'
city_df = df[df['city'] == home_city].iloc[0]['lng']
city_df

# The output is more than just a number
# np.float64 means that the value is a floating-point number
# and it comes from the NumPy library

In [ ]:
# Assign "Bengaluru" as home_city (overwrites previous home_city)
# Find where 'city_ascii' matches home_city
# then extract longitude
# Basically, this code retrieves the longitude of Bengaluru
home_city = 'Bengaluru'

country_df[country_df['city_ascii'] == home_city].iloc[0]['lng']

To locate a particular row or column from a dataframe, Pandas providea `loc[]` and `iloc[]` methods - that allows you to *locate* particular slices of data. Learn about [different indexing methods](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#different-choices-for-indexing) in Pandas. Here we can use `iloc[]` to find the row matching the `home_city` name. Since `iloc[]` uses index, the *0* here refers to the first row.

In [ ]:
home_city = 'Bengaluru'
filtered = country_df[country_df['city_ascii'] == home_city]
filtered.iloc[0]

Now that we have filtered down the data to a single row, we can select individual column values using column names.

In [ ]:
home_city_coordinates = (filtered.iloc[0]['lat'], filtered.iloc[0]['lng'])
home_city_coordinates

## Performing calculations

Let's learn how to do calculations on a dataframe. We can iterate over each row and perform some calculations. But pandas provide a much more efficient way. You can use the `apply()` method to run a function on each row. This is fast and makes it easy to complex computations on large datasets.

The `apply()` function takes 2 arguments. A function to apply, and the axis along which to apply it. `axis=0` means it will be applied to columns and `axis=1` means it will apply to rows.

![](https://github.com/spatialthoughts/python-foundation-web/blob/master/images/python_foundation/pandas_axis.png?raw=1)


We'll need a module called 'geopy' for this. Let's check if it is already installed, and if not, install it

In [ ]:
try:
    import geopy
    print(f"✓ geopy is ready (v{geopy.__version__})")
except ImportError:
    print("geopy not found. Installing...")
    %pip install geopy
    import geopy
    print(f"✓ geopy has been installed and imported (v{geopy.__version__})")

In [ ]:
from geopy import distance # imports distance module from geopy library

def calculate_distance(row): # creates the function; will be applied to each row
    city_coordinates = (row['lat'], row['lng']) # extracts lat/lon; stored in tuple
    return distance.geodesic(city_coordinates, home_city_coordinates).km
    # above line calculates geodesic distanc in km between each row and home city

# below applies dist function to country.df
# apply() runs the function on rows
# results stored in 'result' variable
result = country_df.apply(calculate_distance, axis=1)
result

We can add these results to the dataframe by simply assigning the result to a new column named 'distance'.

In [ ]:
country_df['distance'] = result
country_df

We are done with our analysis and ready to save the results. We can further filter the results to only certain columns.

In [ ]:
filtered = country_df[['city_ascii','distance']]
filtered

Let's rename the `city_ascii` column to give it a more readable name.

In [ ]:
filtered = filtered.rename(columns = {'city_ascii': 'city'})
filtered

Now that we have filtered the original data and computed the distance for between home_city and all cities, we can save the resulting dataframe to a file. Similar to read methods, Pandas has several write methods, such as `to_csv()`, `to_excel()` etc.

Here we will use the `to_csv()` method to write a CSV file. Pandas assigns an index column (unique integer values) to a dataframe by default. We specify `index=False` so that this index is not added to our output.

In [ ]:
# First, we'll set up an output directory and filename
output_dir = 'output' 
output_filename = 'cities_distance_pandas.csv'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, output_filename)

filtered.to_csv(output_path, index=False)
print('Successfully written output file at {}'.format(output_path))

## Exercise

You will notice that the output file contains a row with the `home_city` as well, with a distance of 0, of course. Modify the `filtered` dataframe to remove this row and write the results to a new file named cities_distance_pandas_fixed.csv.

Hint: Use the [Boolean Filtering](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) method we learned earlier to select rows that do not match the `home_city`.

The cell below contains most of the code needed, copied from relevant cells above. Figure out what you need to add to solve this.

In [ ]:
import os
from geopy import distance
import pandas as pd

def calculate_distance(row):
    city_coordinates = (row['lat'], row['lng'])
    return distance.geodesic(city_coordinates, home_city_coordinates).km

data_folder = 'data'
filename = 'worldcities.csv'
path = os.path.join(data_folder, filename)
df = pd.read_csv(path)

home_country = 'India'
home_city = 'Bengaluru'

country_df = df[df['country'] == home_country].copy()
filtered = country_df[country_df['city_ascii'] == home_city]
home_city_coordinates = (filtered.iloc[0]['lat'], filtered.iloc[0]['lng'])

country_df['distance'] = country_df.apply(calculate_distance, axis=1)
filtered = country_df[['city_ascii','distance']]
filtered = filtered.rename(columns = {'city_ascii': 'city'})








----